## Interview Questions

## Overview of this workbook:

We want to count cells across multiple subjects, conditions, brain regions, and experimental conditions. 

### Experimental Overview
You have an excel sheet with information about brain region cell counts. We want to compare the number of active cells across two conditions. Each sheet in the excel sheet represents and is labeled by condition:
-   condition1
-   condition2

We have counted cells in 3 brain regions in three areas: anterior, medial, and posterior (you can think of them as front, middle, and back of each region)
-   region1
-   region2
-   region3

We have also counted cells in these three regions that talk to two other regions:
-   regionA
-   regionB 

We have injected some pretty colors to label projections (red and green) - this means if I inject red into regionA and I see red in region1, I know region1 talks to regionA through neurons that start in region1 and travel to regionA. 

We have also marked and counted which cells were active. 

We also have individual subjects with IDs such as: 
-   1_1
-   2_1
-   2_2
-   2_3
-   6_1
and so on

### Overview of the excel sheet 
You have two columns on each sheet in the excel: 
-   slice
-   count

how to read slice names: area_region_SubjectID_active_colors

### Example slice names

**posterior_region3_1_anterior_regionion1_1_1_active**: this means the count in this row represents the number of cells in a posterior (back) slice of brain region1 from Subject 1_1 of active cells ONLY (that are not green or red, just active) 

**posterior_region3_1_anterior_regionion1_1_1_active_green**: this means all of the same stuff except this count is for active cells that are also green 

**posterior_region3_1_anterior_regionion1_1_1_green**: similarly this means this count is for cells that are only green and NOT active. 

### GOAL

We want to count cells across all subjects and conditions that are active or that talk to regionA (including those that talk to regionB) or regionB (including those talk to regionA). 
Additionally, the experimenter injected green into regionA *most of the time* but not all of the time. So in order to know which brain regions are talking to which we need to map subjects to their injection color pairs.

So with this in mind we need to 
1. map subjects to brain regions by color
2. reformat the data
3. calculate new totals
4. confirm our totals are correct 



In [ ]:
import pandas as pd

# Please debug this cell to find the data file you are looking for and load it in using pandas. 
# Reminder that there are two sheets in the excel file: one for condition1 and one for condition2

file_path = "\cell_counts_full.xlsx"

# Load both sheets into separate DataFrames
sheet = pd.read_excel(file_path)


# Define columbs
sheet1.columns = ["Slice", "Count"]
sheet2.columns = ["Slice", "Count"]


## Humans in action
As all this data is collected and created by human's efforts - it's not always pretty. Our lovely experimenter injected green into regionA *most of the time* but not all of the time. So in order to know which brain regions are talking to which we need to map subjects to their injection color pairs 

In [ ]:
# Define subject-specific mappings for color to brain region in condition 1
subject_mappings_condition1 = {
    '2_1': {'green': 'regionB', 'red': 'regionA'},
    '2_2': {'green': 'regionB', 'red': 'regionA'},
    '2_3': {'green': 'regionB', 'red': 'regionA'},
    '1_2': {'green': 'regionA', 'red': 'regionB'},
    '1_3': {'green': 'regionA', 'red': 'regionB'}
}

# Define the default mapping for condition2
default_mapping_condition2 = {'green': 'regionA', 'red': 'regionB'}



Now lets use the above mapping to map them to the slice names in our excel sheet

In [ ]:
# FUNCTIONS TO EXTRACT RELEVENT INFORMATION FROM SLICE NAME

# Function to extract color from Slice name
def extract_color(slice_value):
    if 'green' in slice_value:
        return 'green'
    elif 'red' in slice_value:
        return 'red'
    elif 'active' in slice_value:
        return 'active'
    return 'Unknown'

# Function to extract the subject number from the Slice column
# There is a bug in this function please look closely 
def extract_subject(slice_value):
    parts = slice_value.split('_')
    if len(parts) >= 3:
        return parts[2]  # 
    return 'Unknown'  # In case the slice format is unexpected

def extract_region(slice_value):
    parts = slice_value.split('_')
    if len(parts) >= 4:
        return parts[3]  # Assuming the region information is in the 4th position
    return 'Unknown'


# Function to extract the region information from the Slice column
def extract_region(slice_value):
    parts = slice_value.split('_')
    if len(parts) >= 4:
        return parts[3]  # Assuming the region information is in the 4th position
    return 'Unknown'  # In case the slice format is unexpected


In [6]:
# Functions to map and label what we know with what the excel sheet has

#Map color to subject based on subject dictionary defined earlier
# There is a bug in this function please look closely 
def map_color(slice_value, color_value, sheet_name):
    parts = slice_value.split('_')
    subject = parts[2]  # Extract subject id from the Slice column
    if sheet_name == 'condition1' and subject in subject_mappings_condition1:
        region_mapping = subject_mappings_condition1[subject]
        return region_mapping.get(color_value, 'Unknown')
    elif sheet_name == 'condition2':
        return default_mapping_condition2.get(color_value, 'Unknown')
    return 'Unknown'

# Define overlaps based on the presence of markers in the Slice column
def determine_overlap(slice_value):
    markers = []
    if 'active' in slice_value:
        markers.append('active')
    if 'green' in slice_value:
        markers.append('green')
    if 'red' in slice_value:
        markers.append('red')
    # Join markers to form the overlap label
    return '_'.join(sorted(markers))

# Function to extract markers from Slice and create projector label
def create_projector(slice_value, subject, sheet_name):
    markers = []
    if 'active' in slice_value:
        markers.append('active')
    if 'green' in slice_value:
        markers.append('green')
    if 'red' in slice_value:
        markers.append('red')
    if sheet_name == 'condition1' and subject in subject_mappings_condition1:
        region_mapping = subject_mappings_condition1[subject]
    else:
        region_mapping = default_mapping_condition2
    combined_markers = []
    for marker in markers:
        if marker in ['green', 'red']:
            region = region_mapping.get(marker, 'Unknown')
            combined_markers.append(region)
        else:
            combined_markers.append(marker)
    return '+'.join(combined_markers) + '+'

In [ ]:
# Process the data for both sheets
def process_data(df, sheet_name):
    df['Slice'] = df['Slice'].astype(str)
    df['Subject'] = df['Slice'].apply(extract_subject)  # Extract the subject number
    df['Region'] = df['Slice'].apply(extract_region)    # Extract the region information
    df['color'] = df['Slice'].apply(extract_color)
    df['CellType'] = df.apply(lambda row: map_color(row['Slice'], row['color'], sheet_name), axis=1)  #if you do not know what a .apply(lamda x) fxn does lets figure that out 
    df['Overlap'] = df['Slice'].apply(determine_overlap)
    df['Projector'] = df.apply(lambda row: create_projector(row['Slice'], row['Subject'], sheet_name), axis=1)
    return df

# Apply the updated function to both sheets
sheet1_processed = process_data(sheet1, 'condition1')
sheet2_processed = process_data(sheet2, 'condition2')

# Print the processed data (or save to new Excel file, etc.)
print("Processed data from condition1:")
print(sheet1_processed)

print("Processed data from condition2:")
print(sheet2_processed)

An example of how we started and where we ened up:

In [ ]:
example_slice = 'anterior_region1_1_1_red_green_active.png'
example_row = sheet1_processed[sheet1_processed['Slice'] == example_slice]
if not example_row.empty:
    example_projector = example_row['Projector'].values[0]
    print(f"Projector for {example_slice}: {example_projector}")

BIG BUG: 

For the condition1 the counts are off. Specifically for multiple projectors 
condition1 totals. The correct counts are as follows.
   - regionA+ = 7224
   - regionB+ = 6049
   - regionB+regionA+ = 4670 
   - active = 5308 
   - active+regionA+ = 5080 
   - active+regionB+ = 4641 
   - active+regionA+regionB+ = 4166


   How would you go about trying to figure out the issue? 

In [ ]:
projector_counts = pd.concat([sheet1_processed.groupby('Projector')['Count'].sum().reset_index().assign(sheet='condition1'),
                             sheet2_processed.groupby('Projector')['Count'].sum().reset_index().assign(sheet='condition2')])

print("Total counts for each projector by sheet:")
print(projector_counts)